In [ ]:
import numpy as np
from os import path
import matplotlib.pyplot as plt

In [ ]:
### Load data

# imresps.npy is of shape (1573, 2, 15363), where 1573 is number of images, 2 repeats each, and 15363 neurons recorded
# stimids.npy has the image id (matching the image dataset ~selection1866~) for each stimulus number, 
# so of you want to see what image was presented on imresps[502] you would check stim_ids[502]

PATH_TO_DATA = '../../data/neural'

imresps = np.load(path.join(PATH_TO_DATA, 'imresps.npy'))
stimids = np.load(path.join(PATH_TO_DATA, 'stimids.npy'))

print(imresps.shape) # (1573, 2, 15363)
print(stimids.shape) # (1573,)

In [ ]:
def compute_signal_related_variance(resp_a, resp_b, mean_center=True):
    """
    compute the fraction of signal-related variance for each neuron,
    as per Stringer et al Nature 2019. Cross-validated by splitting
    responses into two halves. Note, this only is "correct" if resp_a
    and resp_b are *not* averages of many trials.

    Args:
        resp_a (ndarray): n_stimuli, n_cells
        resp_b (ndarray): n_stimuli, n_cells

    Returns:
        fraction_of_stimulus_variance: 0-1, 0 is non-stimulus-caring, 1 is only-stimulus-caring neurons
        stim_to_noise_ratio: ratio of the stim-related variance to all other variance
    """
    if len(resp_a.shape) > 2:
        # if the stimulus is multi-dimensional, flatten across all stimuli
        resp_a = resp_a.reshape(-1, resp_a.shape[-1])
        resp_b = resp_b.reshape(-1, resp_b.shape[-1])
    ns, nc = resp_a.shape
    if mean_center:
        # mean-center the activity of each cell
        resp_a = resp_a - resp_a.mean(axis=0)
        resp_b = resp_b - resp_b.mean(axis=0)
    
    # compute the cross-trial stimulus covariance of each cell
    # dot-product each cell's (n_stim, ) vector from one half
    # with its own (n_stim, ) vector on the other half

    covariance = (resp_a * resp_b).sum(axis=0) / ns

    # compute the variance of each cell across both halves
    resp_a_variance = (resp_a**2).sum(axis=0) / ns
    resp_b_variance = (resp_b**2).sum(axis=0) / ns
    total_variance = (resp_a_variance + resp_b_variance) / 2

    if np.any(total_variance < 1e-12):
        print(f"Warning: Near-zero total variance for neurons: {np.where(total_variance < 1e-12)[0]}")

    # compute the fraction of the total variance that is
    # captured in the covariance
    fraction_of_stimulus_variance = covariance / total_variance

    # if you want, you can compute SNR as well:
    stim_to_noise_ratio = fraction_of_stimulus_variance / (
        1 - fraction_of_stimulus_variance
    )

    return fraction_of_stimulus_variance, stim_to_noise_ratio

In [ ]:
### Compute the null distribution of SRV values for all neurons

# imresps shape = (1573, 2, 15363)
# responses in imresps shape = (2, 15363)
num_stimuli = imresps.shape[0] # 1573
num_repeats = imresps.shape[1] # 2
num_neurons = imresps.shape[2] # 15363
n_shuffles = 100

null_srv_all_neurons = [] # shape (n_shuffles, num_neurons)

for _ in range(n_shuffles):
    # Shuffle stimulus indices *twice* to create two independent splits!
    shuffled_indices_A = np.random.permutation(num_stimuli)
    shuffled_indices_B = np.random.permutation(num_stimuli)

    # Now for the splits, we can just use fixed repeat indices, 
    # because for each split, at index N the responses correspond to different stimuli
    # e.g. split_A = [ stim_100_repeat_1, stim_2_repeat_1, stim_19_repeat_1, ... ]
    # e.g. split_B = [ stim_543_repeat_2, stim_345_repeat_2, stim_3_repeat_2, ... ]
    split_A = imresps[shuffled_indices_A, 0, :]
    split_B = imresps[shuffled_indices_B, 1, :]

    # Compute SRV for the shuffled data
    fraction_of_stimulus_variance, _ = compute_signal_related_variance(split_A, split_B)
    null_srv_all_neurons.append(fraction_of_stimulus_variance)

null_srv_all_neurons = np.array(null_srv_all_neurons)
null_srv_all_neurons.shape # (100, 15363)

print(null_srv_all_neurons[0])
print(null_srv_all_neurons[33])

# e.g. if neuron_index = 0, it will plot the SRV value for neuron 0 across all shuffles
neuron_index = 0
plt.hist([srv[neuron_index] for srv in null_srv_all_neurons], bins=100, color='blue', alpha=0.7)
plt.xlabel("Fraction of Stimulus-Related Variance (SRV)")
plt.ylabel("Number of Shuffles")
plt.title(f"Null Distribution of SRV for Neuron {neuron_index}")
plt.show()

In [ ]:
### Compute the real SRV for each neuron

# split_A_real = imresps[:, 0, :] # First repeat for each stimulus
# split_B_real = imresps[:, 1, :] # Second repeat for each stimulus

split_A, split_B = [], []
for responses in imresps: # responses shape: (2, n_neurons)
    indices = np.random.permutation(2) # Randomly shuffle [0, 1]
    split_A.append(responses[indices[0]]) # Assign one repeat to split_A
    split_B.append(responses[indices[1]]) # Assign the other to split_B

split_A = np.array(split_A)  # Shape: (n_stimuli, n_neurons)
split_B = np.array(split_B)  # Shape: (n_stimuli, n_neurons)

# Compute SRV for real data
real_srv_all_neurons, stim_to_noise_ratio = compute_signal_related_variance(split_A, split_B)

print(real_srv_all_neurons)
print(stim_to_noise_ratio)

print("Real SRV shape:", real_srv_all_neurons.shape) # Should be (15363,)

plt.hist(real_srv_all_neurons, bins=100, color='blue', alpha=0.7)
plt.xlabel("Fraction of Stimulus-Related Variance (SRV)")
plt.ylabel("Number of Shuffles")
plt.title(f"Null Distribution of SRV for Neuron {neuron_index}")
plt.show()

In [ ]:
### Filter neurons whose real SRV is in the top 90th percentile of its null distribution

# This gives the 90th-percentile SRV value of the null distribution for each neuron
# In other words the threshold for each neuron to be considered reliable
# e.g. if neuron 0 has a null distribution of SRVs across 10 shuffles 
# [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], the threshold would be 0.9
top_99th_percentile_null = np.percentile(null_srv_all_neurons, 99, axis=0)
print(top_99th_percentile_null) # [0.03651716 0.03126347 0.03325775 ... 0.02738261 0.03546677 0.0333109 ]

# Get indices of reliable neurons
reliable_neuron_indices = np.where(real_srv_all_neurons >= top_99th_percentile_null)[0]

# Print results
print(f"Number of reliable neurons: {len(reliable_neuron_indices)}") # 5654
print(f"Indices of reliable neurons: {reliable_neuron_indices}") # [   14    29    48 ... 15357 15358 15360]

plt.hist(real_srv_all_neurons, bins=100, color='red', alpha=0.7)
plt.hist(real_srv_all_neurons[reliable_neuron_indices], bins=100, color='blue', alpha=0.7)
plt.xlabel("Fraction of Stimulus-Related Variance (SRV)")
plt.ylabel("Number of Shuffles")
plt.title("All Neurons: SRV all vs. SRV reliable")
plt.show()

plt.hist(real_srv_all_neurons[reliable_neuron_indices], bins=100, color='blue', alpha=0.7)
plt.xlabel("Fraction of Stimulus-Related Variance (SRV)")
plt.ylabel("Number of Neurons")
plt.title("SRV Distribution for Reliable Neurons")
plt.show()

In [ ]:
### Load and preprocess images

import os
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import Normalize, Compose, Resize, CenterCrop
import torch
from torch.utils.data import TensorDataset
from torchvision import utils as torch_utils
 
PATH_TO_DATA = '../../data/selection1866'

file_list = sorted(f for f in os.listdir(PATH_TO_DATA) if f.endswith('.mat'))
stim_ids = stimids.astype(int)

print(stim_ids)
print(stimids)

transform = Compose([
    Resize(96), # Resize shortest edge to 96 (cut off the rightmost part of the image)
    CenterCrop((96, 96)), # Crop to (96, 96)
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), # !! Normalize expects input is already in the range [0, 1]
])

img_tensors, labels = [], []

print('List:', file_list)

# we have 1866 images here, but the neural response data only uses 1573 of them
# because some ~300 images didn't have two repeats, so were disposed
# therefore we filter the full set here so that we only use the relevant 1573
for stim_id in stim_ids:
    filename = 'img' + str(stim_id) + '.mat'
    data = loadmat(os.path.join(PATH_TO_DATA, filename))

    img = data['img'][:, :500] # Take leftmost part of the image
    rgb_img = np.stack([img] * 3, axis=-1) # Convert grayscale to RGB for SimCLR
    tensor = torch.tensor(rgb_img, dtype=torch.float32).permute(2, 0, 1) # Shape (C, H, W)
    
    # Min-max scale the tensor to [0, 1]
    tensor_min = tensor.min()
    tensor_max = tensor.max()
    tensor = (tensor - tensor_min) / (tensor_max - tensor_min)

    # Clamp to [0, 1] to ensure no outliers due to numerical precision
    tensor = torch.clamp(tensor, 0.0, 1.0)

    transformed_tensor = transform(tensor) # Normalize and resize for SimCLR
    img_tensors.append(transformed_tensor)
    labels.append(stim_id)

image_dataset = TensorDataset(torch.stack(img_tensors), torch.tensor(labels))

images, labels = image_dataset.tensors
print("Processed image labels (stim id):", labels[:30])
print("Stim IDs from neural data:", stim_ids[:30])
print("Processed dataset shape:", images.shape) # (N, C, 96, 96)
print(f"Min pixel value (processed): {torch.min(images)}")
print(f"Max pixel value (processed): {torch.max(images)}")

# Show a sample of processed images
img_grid = torch_utils.make_grid(images[:12], nrow=6, normalize=True, pad_value=0.9)
img_grid = img_grid.permute(1, 2, 0).numpy()
plt.figure(figsize=(10, 5))
plt.title('Processed images: sample')
plt.imshow(img_grid)
plt.axis('off')
plt.show()
plt.close()

filename = 'img20.mat'
data = loadmat(os.path.join(PATH_TO_DATA, filename))
img = data['img'][:, :500]

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.imshow(img, cmap='gray')  # Adjust cmap as needed ('viridis', 'jet', etc.)
plt.colorbar(label="Pixel Intensity")
plt.title("Rendered Image")
plt.axis("off")  # Hide axis for better visualization
plt.show()

In [ ]:
### Run images through a pretrained SimCLR model and extract features

import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm.notebook import tqdm
from typing import Dict
from torch.utils.data import Dataset
import urllib.request
from urllib.error import HTTPError
from collections import defaultdict

class SimCLR(nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()

        # Base ResNet18 backbone (pretrained=False, because we load custom weights later, from the SimCLR checkpoint file)
        self.convnet = torchvision.models.resnet18(pretrained=False)
        
        # This is the projection head, only needed during training. For downstream tasks it is disposed of
        # and the final linear layer output is used (Chen et al., 2020) 
        self.convnet.fc = nn.Sequential(
            nn.Linear(self.convnet.fc.in_features, 4 * hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(4 * hidden_dim, hidden_dim)
        )

        self.intermediate_layers_to_capture =[]
        self.intermediate_layer_features = {}
        self.num_workers = os.cpu_count()
        self.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

    def load_pretrained(self):
        """
        Load pretrained SimCLR weights
        """
        base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial17/"
        models_dir = "../../models"
        pretrained_simclr_filename = "SimCLR.ckpt"
        pretrained_simclr_path = os.path.join(models_dir, pretrained_simclr_filename)
        os.makedirs(models_dir, exist_ok=True)

        # Check whether the pretrained model file already exists locally. If not, try downloading it
        file_url = base_url + pretrained_simclr_filename
        if not os.path.isfile(pretrained_simclr_path):
            print(f"Downloading pretrained SimCLR model {file_url}...")
            try:
                urllib.request.urlretrieve(file_url, pretrained_simclr_path)
            except HTTPError as e:
                print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

        print(f"Already downloaded pretrained model: {file_url}")

        # Load pretrained model
        checkpoint = torch.load(pretrained_simclr_path, map_location=self.device)
        self.load_state_dict(checkpoint['state_dict'])
        self.to(self.device)
        self.eval()
    
    def set_intermediate_layers_to_capture(self, layers):
        """
        Register hooks to capture features from intermediate layers
        """
        # Just check the layers specified are actually in the convnet
        top_level_block_layers = [name for name, _ in self.convnet.named_children()]
        if not all(layer in top_level_block_layers for layer in layers):
            print('You have specified convnet layers that are not top-level blocks - make sure your layer names are valid')
        
        self.intermediate_layers_to_capture = layers
        intermediate_layer_features = {}

        def get_hook(layer_name):
            def hook(module, input, output):
                intermediate_layer_features[layer_name] = output.detach()
            return hook

        for layer_name in layers:
            layer = dict([*self.convnet.named_modules()])[layer_name]
            layer.register_forward_hook(get_hook(layer_name))

        self.intermediate_layer_features = intermediate_layer_features

    @torch.no_grad()
    def extract_features(self, dataset: Dataset) -> Dict[str, torch.Tensor]:
        """
        Run the pretrained SimCLR model on the image data, and capture features from final layer and intermediate layers.

        Args:
            dataset (Dataset): A PyTorch Dataset containing input images and labels. The image data should have shape (N, C, H, W)

        Returns:
            Dict[str, torch.Tensor]: A dictionary containing:
                - Intermediate layer features as tensors.
                - Final layer features under 'final_layer'.
                - Labels under 'labels'.
            Features from a given layer has shape (N, F) where N is num images, F is number of features - flattened version of (C, H, W).
        """
        self.convnet.fc = nn.Identity()  # Removing projection head g(.)
        self.eval()
        self.to(self.device)
        
        # Encode all images
        data_loader = DataLoader(dataset, batch_size=64, num_workers=self.num_workers, shuffle=False, drop_last=False)
        feats, labels, intermediate_features = [], [], {layer: [] for layer in self.intermediate_layers_to_capture}

        for batch_idx, (batch_imgs, batch_labels) in enumerate(tqdm(data_loader)):
            batch_imgs = batch_imgs.to(self.device)
            batch_feats = self.convnet(batch_imgs)
            
            feats.append(batch_feats.detach().cpu())
            labels.append(batch_labels)

            # Collect intermediate layer outputs
            for layer in self.intermediate_layers_to_capture:
                # Final linear layer outputs a 2d tensor; but intermediate layers don't, so we flatten them (ready for PCA etc.)
                layer_output_flattened = self.intermediate_layer_features[layer].view(self.intermediate_layer_features[layer].size(0), -1) 
                intermediate_features[layer].append(layer_output_flattened.cpu())
        
        # Concatenate results for each layer
        feats = torch.cat(feats, dim=0)
        labels = torch.cat(labels, dim=0)
        intermediate_features = {layer: torch.cat(intermediate_features[layer], dim=0) for layer in self.intermediate_layers_to_capture}

        # Debugging log after concatenation
        print("✅ Feature extraction complete. Final feature shapes:")
        print(f"Final layer: {feats.shape}")
        for layer, feature in intermediate_features.items():
            print(f"{layer}: {feature.shape}")  # Check final stored shape

        return {**intermediate_features, 'final_layer': feats, 'labels': labels}

intermediate_layers = ['layer1', 'layer2', 'layer3', 'layer4']

sim_clr = SimCLR()
sim_clr.load_pretrained()
sim_clr.set_intermediate_layers_to_capture(intermediate_layers)
feats = sim_clr.extract_features(image_dataset)

for layer in ["layer1", "layer2", "layer3", "layer4"]:
    if layer in feats:
        variance = np.var(feats[layer].numpy())
        print(f"{layer} variance: {variance:.6f}")

# Our original images are grayscale, but SimCLR expects 3-channel RGB input.
# To meet this requirement, we duplicated the grayscale values across all three RGB channels.
# However, for PCA, we only need a single channel, so we extract just the first channel (Red).
flattened_images = images[:, 0, :, :].view(images.shape[0], -1) # shape: [1573, 50176] (1573 images, 224x224 pixels)

layer1_feats = feats['layer1'] # Shape: torch.Size([1573, 200704]) (n_images, n_features)
layer2_feats = feats['layer2']
layer3_feats = feats['layer3']
layer4_feats = feats['layer4']
final_layer_feats = feats['final_layer'] # Shape: torch.Size([1573, 512])

print('flattened_images shape', flattened_images.shape)
print('layer1 shape', layer1_feats.shape)
print('final layer shape', final_layer_feats.shape)

In [ ]:
### 1. Predict PC of neural data (separate model predicts each PC), e.g. predict PC1 neural data from features of layer2
### 2. Feature visualisation of most important features for PC, for layer, each neuron.

from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

images_representations = {
    'raw_pixels': flattened_images,
    'layer1': layer1_feats,
    'layer2': layer2_feats,
    'layer3': layer3_feats,
    'layer4': layer4_feats,
    'fc': final_layer_feats,
}

# ===================================
# Filter only the top Y neurons (SRV)
# ===================================
num_neurons = 500
reliable_srv_scores = real_srv_all_neurons[reliable_neuron_indices]
sorted_indices = np.argsort(reliable_srv_scores)[::-1]
most_reliable_neurons = reliable_neuron_indices[sorted_indices[:num_neurons]]
highest_srv_scores = real_srv_all_neurons[most_reliable_neurons]
neural_responses = imresps[:, :, most_reliable_neurons]
neural_responses_mean = neural_responses.mean(axis=1)

assert most_reliable_neurons.shape[0] == num_neurons, "Mismatch in neuron selection!"
print("Dimensionality of neural responses:", neural_responses_mean.shape)
print("Top 500 reliable neuron indices:", most_reliable_neurons[:10])
print("Corresponding SRV scores:", highest_srv_scores[:10])
print("Top 500 neural responses shape:", neural_responses.shape) # (1573, 2, 500)
print("Averaged top 500 neural responses shape:", neural_responses_mean.shape) # (1573, 500)

# ===================================
# Get PCs of neural data
# ===================================
pca = PCA(10)
neural_data_pcs = pca.fit_transform(neural_responses_mean)
pc1_neural_data = neural_data_pcs[:, 0]

# ===================================
# Regression from image representation to first PC of neural data 
# ===================================
for k in images_representations:
    X_train, X_test, y_train, y_test = train_test_split(images_representations[k], pc1_neural_data, test_size=0.2, random_state=42)

    ridge = Ridge(alpha=1000)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f"R^2 Score for {k}: {r2:.4f}")

# R^2 Score for raw_pixels: 0.0029
# R^2 Score for layer1_features: 0.4033
# R^2 Score for layer2_features: 0.4820
# R^2 Score for layer3_features: 0.4715
# R^2 Score for layer4_features: 0.3455
# R^2 Score for final_layer_features: 0.0675

In [ ]:
### Summary of results for each layer

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt

pc_index = 0 # Index of the principal component to predict

results = {}
for k in images_representations:
    # ================================
    # Step 1: Split data into training and test sets
    # ================================
    X_train, X_test, y_train_full, y_test_full = train_test_split(images_representations[k], neural_responses_mean, test_size=0.2, random_state=42)

    # ================================
    # Step 2: Fit PCA only on the training set
    # ================================
    pca = PCA(n_components=10)
    y_train_pcs = pca.fit_transform(y_train_full)  # Fit PCA on training neural data
    y_test_pcs = pca.transform(y_test_full)  # Transform test neural data

    # Extract PC as target variable
    y_train = y_train_pcs[:, pc_index] 
    y_test = y_test_pcs[:, pc_index]

    # ================================
    # Step 3: Perform Cross-Validation for Ridge Regression
    # ================================
    alphas = np.array([0.1, 1, 10, 100, 1000, 100000, 1000000])
    best_alpha = None
    best_score = -np.inf

    kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

    for alpha in alphas:
        ridge = Ridge(alpha=alpha)
        scores = cross_val_score(ridge, X_train, y_train, cv=kf, scoring="r2")
        mean_score = scores.mean()
        
        if mean_score > best_score:
            best_score = mean_score
            best_alpha = alpha

    print(f"Best alpha from CV: {best_alpha:.4f}, Mean CV R²: {best_score:.4f}")

    # ================================
    # Step 4: Train Ridge Regression with Best Alpha
    # ================================
    ridge = Ridge(alpha=best_alpha)
    ridge.fit(X_train, y_train)

    # ================================
    # Step 5: Evaluate Model on Test Set
    # ================================
    y_pred = ridge.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f"Test R² Score: {r2:.4f}")
    results.update({k: {"best_alpha": best_alpha, "mean_cv_r2": best_score, "test_r2": r2, "ridge": ridge}})

    # ================================
    # Step 6: Plot Predictions vs True PC1 Values
    # ================================
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.xlabel(f"True PC{pc_index+1} of Neural Data")
    plt.ylabel(f"Predicted PC{pc_index+1}")
    plt.title(f"Ridge Regression: {k} → PC{pc_index+1} of Neural Data")
    plt.axline((0, 0), slope=1, color="red", linestyle="--", label="Ideal Fit")
    plt.legend()
    plt.show()

print(results)

In [ ]:
### Plot summary of results across all 

layers = list(results.keys())
mean_cv_r2 = [results[layer]['mean_cv_r2'] for layer in layers]
test_r2 = [results[layer]['test_r2'] for layer in layers]
best_alphas = [results[layer]['best_alpha'] for layer in layers]

plt.figure(figsize=(10, 5))
bar_width = 0.4
x = np.arange(len(layers))
bars1 = plt.bar(x - bar_width/2, mean_cv_r2, bar_width, label="Mean CV R²")
bars2 = plt.bar(x + bar_width/2, test_r2, bar_width, label="Test R² Score")

for i in range(len(layers)):
    plt.text(x[i] - bar_width/2, mean_cv_r2[i] + 0.02, f"α={int(best_alphas[i])}", 
             ha="center", fontsize=10, fontweight="bold")

plt.xlabel("Feature Representation Layer")
plt.ylabel("R² Score")
plt.title(f"Ridge Regression to PC{pc_index+1} of Neural Data (average response of each of the 500 most stimulus-responsive neurons)")
plt.xticks(ticks=x, labels=layers, rotation=45)
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
### Ridge regression feature coefficients
import numpy as np
import matplotlib.pyplot as plt

layers_feats_top_features = {
    'raw_pixels': { 'feats': flattened_images, 'top_features': [] },
    'layer1': { 'feats': layer1_feats, 'top_features': [] },
    'layer2': { 'feats': layer2_feats, 'top_features': [] },
    'layer3': { 'feats': layer3_feats, 'top_features': [] },
    'layer4': { 'feats': layer4_feats, 'top_features': [] },
    'fc': { 'feats': final_layer_feats, 'top_features': [] },
}

for layer in images_representations:
    ridge = results[layer]['ridge']

    feature_importance = np.abs(ridge.coef_)
    sorted_indices = np.argsort(feature_importance)[::-1]
    top_features = sorted_indices[:20] # Select top 20 important features

    layers_feats_top_features[layer]['top_features'] = top_features

    plt.figure(figsize=(10, 5))
    plt.bar(range(len(top_features)), feature_importance[top_features])
    plt.xticks(range(len(top_features)), top_features, rotation=90)
    plt.xlabel("Feature Index")
    plt.ylabel("Absolute Coefficient Magnitude")
    plt.title(f"[{layer}]: Feature Importance for Predicting PC{pc_index+1} ({layer})")
    plt.show()

print(layers_feats_top_features)

In [ ]:
### feature visualization: generate synthetic images optimized to maximally activate a given feature in a given layer
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

for k in layers_feats_top_features:
    if k == 'raw_pixels':
        continue

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load pretrained SimCLR model
    sim_clr = SimCLR()
    sim_clr.load_pretrained()
    sim_clr.set_intermediate_layers_to_capture([k])  # Capture layer2 features
    sim_clr.eval()
    sim_clr.to(device)

    # Top neurons (from Ridge regression feature importance analysis)
    top_features = layers_feats_top_features[k]['top_features'][:5]

    # Optimization settings
    num_iterations = 200  # Number of gradient ascent steps
    learning_rate = 0.05  # Step size
    image_size = 224  # Image resolution

    # Create synthetic images
    fig, axes = plt.subplots(1, len(top_features), figsize=(15, 5))

    # Register hook only once
    intermediate_features = {}

    def hook_fn(module, input, output):
        intermediate_features[k] = output

    layer = dict([*sim_clr.convnet.named_modules()])[k]
    hook_handle = layer.register_forward_hook(hook_fn)

    for i, neuron in enumerate(top_features):
        # Start with random noise image
        # synthetic_image = torch.randn(1, 3, image_size, image_size, device=device, requires_grad=True)
        synthetic_image = torch.randn(1, 1, image_size, image_size, device=device, requires_grad=True) # greyscale
        
        # Optimizer
        optimizer = torch.optim.Adam([synthetic_image], lr=learning_rate, weight_decay=1e-6)  # Added weight decay

        for _ in range(num_iterations):
            optimizer.zero_grad()

            # Convert grayscale image into 3-channel format for SimCLR input
            grayscale_image_3channel = synthetic_image.repeat(1, 3, 1, 1)  # (1, 1, H, W) → (1, 3, H, W)

            # Forward pass through SimCLR to extract layer2 features
            _ = sim_clr.convnet(grayscale_image_3channel)  # Forward pass
            features = intermediate_features[k]  # Get extracted layer2 features

            # Ensure neuron index is valid
            neuron_idx = min(neuron, features.shape[1] - 1)  # Avoid out-of-range errors
            loss = -features[0, neuron_idx].mean()

            # Regularization (Total Variation Loss for smooth images)
            tv_loss = torch.sum(torch.abs(synthetic_image[:, :, :, :-1] - synthetic_image[:, :, :, 1:])) + \
                    torch.sum(torch.abs(synthetic_image[:, :, :-1, :] - synthetic_image[:, :, 1:, :]))
            loss += 0.0001 * tv_loss

            # Backpropagation & update image
            loss.backward()
            optimizer.step()

            # Normalize image values
            synthetic_image.data = torch.clamp(synthetic_image.data, -1, 1)

        # Convert image to NumPy format
        # img_np = synthetic_image.detach().cpu().squeeze().permute(1, 2, 0).numpy()
        img_np = synthetic_image.detach().cpu().squeeze().numpy() # Keep grayscale shape (H, W)
        img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())  # Normalize to [0,1]

        # Display result
        axes[i].imshow(img_np, cmap="gray")
        axes[i].axis("off")
        axes[i].set_title(f"Feature {neuron}")

    # Remove hook to prevent memory leak
    hook_handle.remove()

    fig.suptitle(f"Feature Visualization for {k} (Features most predictive of PC{pc_index+1} of neural data)")
    plt.show()

In [ ]:
### For a given feature from a given layer, show the most activating and least activating images

import torch
import matplotlib.pyplot as plt

print(layer4_feats.shape)

layer_name = 'layer3'
chosen_feature = 181

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sim_clr = SimCLR()
sim_clr.load_pretrained()
sim_clr.set_intermediate_layers_to_capture([layer_name])
sim_clr.eval()
sim_clr.to(device)

image_activations = []

print('IMAGE DATASET LEN:', len(image_dataset))

for img_idx, img in enumerate(image_dataset):  
    img_tensor = img[0].unsqueeze(0).to(device)
    _ = sim_clr.convnet(img_tensor)
    features = sim_clr.intermediate_layer_features[layer_name]

    # Flatten features to match `layer4_feats.shape`
    features = features.view(1, -1)  # (1, 512*3*3) = (1, 4608)

    # Get activation of chosen neuron (mean activation across spatial locations)
    activation = features[0, chosen_feature].mean().item()
    image_activations.append((img_idx, activation))

image_activations.sort(key=lambda x: x[1], reverse=True) 

# Get top 5 and bottom 5 activating images
top_images = [image_dataset[i[0]][0] for i in image_activations[:10]] 
bottom_images = [image_dataset[i[0]][0] for i in image_activations[-10:]]

def plot_images(top_images, bottom_images, title):
    fig, axes = plt.subplots(4, 5, figsize=(15, 12))  # 4 rows, 5 columns

    # Plot top 10 activating images (first 2 rows)
    for i, img in enumerate(top_images):
        img_np = img.numpy()[0]  # Take only the first channel (grayscale)
        row, col = divmod(i, 5)  # Get row & column index
        axes[row, col].imshow(img_np, cmap="gray")
        axes[row, col].axis("off")
        axes[row, col].set_title(f"Top {i+1}")

    # Plot bottom 10 activating images (last 2 rows)
    for i, img in enumerate(bottom_images):
        img_np = img.numpy()[0]  # Take only the first channel (grayscale)
        row, col = divmod(i, 5)  # Get row & column index
        axes[row + 2, col].imshow(img_np, cmap="gray")  # Offset by 2 rows
        axes[row + 2, col].axis("off")
        axes[row + 2, col].set_title(f"Bottom {i+1}")

    fig.suptitle(title)
    plt.tight_layout()
    plt.show()

plot_images(top_images, bottom_images, f"Top & Bottom Activating Images for Feature {chosen_feature}")

In [ ]:
### Let's look at how the mouse visual cortex neurons respond to certain images

import matplotlib.pyplot as plt
import numpy as np

print('IMAGE DATASET LEN:', len(image_dataset))

# Get the top 10 image indexes based on the sorted activations, and plot images
top_indices = [img_idx for img_idx, _ in image_activations[:10]]
bottom_indices = [img_idx for img_idx, _ in image_activations[-10:]]
print("Top activating image indexes:", top_indices)

fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, idx in enumerate(top_indices):
    img_tensor = image_dataset[idx][0]
    img_np = img_tensor.numpy()[0]
    
    row, col = divmod(i, 5)
    axes[row, col].imshow(img_np, cmap="gray")
    axes[row, col].axis("off")
    axes[row, col].set_title(f"Idx {idx}")

plt.tight_layout()
plt.show()

### We want to plot the average neural response of each neuron to the top 10 activating images vs. non-activating images vs. bottom 10 activating images vs. all images

print('NEURAL RESPONSES SHAPE:', neural_responses.shape)

# Extract neural responses for flying bird images
flying_bird_responses = neural_responses[top_indices, :] # shape: (num_flying_bird_images, num_neurons)

print('FLYING BIRD RESPONSES SHAPE:', flying_bird_responses.shape)

# Compute the average response of each neuron across the flying bird images
avg_responses = flying_bird_responses.mean(axis=(0, 1))  # shape becomes (500,)

print('AVERAGE RESPONSES SHAPE:', avg_responses.shape)

# Optionally, compute the average response for non-flying bird images for comparison:
all_indices = np.arange(len(image_dataset))
non_bird_indices = np.setdiff1d(all_indices, top_indices)
non_bird_responses = neural_responses[non_bird_indices, :]
avg_responses_non_bird = non_bird_responses.mean(axis=(0, 1))  # shape becomes (500,)

bottom_images_responses = neural_responses[bottom_indices]
avg_bottom_responses = bottom_images_responses.mean(axis=(0, 1))

num_neurons = avg_responses.shape[0] # should be 500

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.bar(np.arange(num_neurons), avg_responses, color='green')
plt.xlabel("Neuron Index")
plt.ylabel("Average Neural Response")
plt.title("Average Response to Flying Bird Images")

plt.subplot(1, 2, 2)
plt.bar(np.arange(num_neurons), avg_responses_non_bird, color='gray')
plt.xlabel("Neuron Index")
plt.ylabel("Average Neural Response")
plt.title("Average Response to Non-Bird Images")

plt.tight_layout()
plt.show()

plt.figure(figsize=(14, 6))
plt.plot(np.arange(num_neurons), avg_responses, color='green', label="Top 10 Activating Images")
plt.plot(np.arange(num_neurons), avg_bottom_responses, color='red', label="Bottom 10 Activating Images")
plt.plot(np.arange(num_neurons), neural_responses.mean(axis=(0, 1)), color='blue', label="All Images")
plt.xlabel("Neuron Index")
plt.ylabel("Average Neural Response")
plt.title("Average Neural Response: Overlay of Flying Bird vs. Non-Bird Images")
plt.legend()
plt.tight_layout()
plt.show()